In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RAG')

Mounted at /content/drive


In [2]:
pip install sentence-transformers faiss-cpu


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import json
import numpy as np
from tqdm import tqdm
import os

In [4]:
CHUNK_FILE = "chunks/chunks.jsonl"
EMBEDDING_MODEL = "bkai-foundation-models/vietnamese-bi-encoder"

texts = []
metas = []

with open(CHUNK_FILE, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        texts.append(item["text"])
        metas.append({
            "source_file": item.get("source_file"),
            "category": item.get("category"),
            "chunk_id": item.get("chunk_id")
        })

print(f"Tổng chunk: {len(texts)}")


Tổng chunk: 1569


In [5]:
model = SentenceTransformer(EMBEDDING_MODEL)
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)
print(f"Embedding shape: {embeddings.shape}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Embedding shape: (1569, 768)


In [6]:
# Tạo FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [7]:
# Lưu file
import pickle

os.makedirs("embeddings", exist_ok=True)

with open("embeddings/faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)

with open("embeddings/metadata.jsonl", "w", encoding="utf-8") as f:
    for meta in metas:
        f.write(json.dumps(meta, ensure_ascii=False) + "\n")


In [10]:
query = "Ai là hiệu trưởng của trường Đại học Công Nghệ?"
query_embedding = model.encode([query])
_, indices = index.search(np.array(query_embedding), k=5)

for idx in indices[0]:
    print(f"{texts[idx]}\n---")


Hiệu trưởng Trường ĐH Khoa học và Tự nhiên. 8. . Ông Hoàng Anh Tuấn. Hiệu trưởng Trường ĐH Khoa học Xã hội và Nhân văn. 9. . Ông. Đỗ Tuấn Minh. Chủ tịch Hội đồng Trường ĐH. Ngoại ngữ. 10. . Ông Chử Đức Trình. Hiệu trưởng Trường ĐH Công nghệ. 1. 1. . Ông. Nguyễn Quý Thanh. Hiệu trưởng Trường ĐH Giáo dục. 12. Ông Nguyễn Trúc Lê. Chủ tịch Hội đồng Trường ĐH Kinh tế. 13. . Ông Furuta Motoo. Hiệu trưởng Trường ĐH Việt Nhật. 14. . Ông Lê Ngọc Thành. Hiệu trưởng Trường ĐH Y Dược. 15.
---
Quốc hội nước Cộng hòa xã hội chủ nghĩa Việt Nam khóa XIII, kỳ họp thứ 3 thông qua ngày 18 tháng 6 năm 2012. Luật Giáo dục Đại học. Chủ tịch Quốc hội Nguyễn Sinh Hùng. Quyết định. ngày 13/5/2008. Về việc ban hành Quy định về chức năng và nhiệm vụ của Văn phòng, các ban chức năng và một số đơn vị trực thuộc Đại học Quốc gia Hà Nội có chức năng quản lý đào tạo sau đại học và kiểm định chất lượng. (Hết hiệu lực). Giám đốc ĐHQGHN Mai Trọng Nhuận. uyết định. ngày 01/10/2001.
---
Việc phát hiện, đào tạo, 